# Feature Extraction from Time Series Data Using a Neural Network

Currently not using pycbc data yet as constructing the model

TODO:
- Generate simple data -> Done
- Add noise -> Done
- Build NN model for parameter estimation
- Training loop -> Done I think
- Testing of trained model
- Update to pycbc noise
- Update Model to do parameter prosterior prediction, look at dingo
- Update to pycbc data
- Finished?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# just generating some datasets for the time being
data_samples = 1000
mean = 10
std = 2
no_time_steps = 4000

amplitudes = np.random.normal(mean, std, data_samples)

time_steps = np.linspace(0,120,no_time_steps)

signals = np.array([np.sin(time_steps)*a for a in amplitudes])

# adding noise, just gaussian for the time being
noise_mean = 1
noise_std = 2
noised_signals = np.array([x + np.random.normal(noise_mean, noise_std, no_time_steps) for x in signals])

# convert to pytorch datasets

X = torch.FloatTensor(noised_signals)
y = torch.FloatTensor(amplitudes)


data = TensorDataset(X,y)

train_data, val_data, test_data = random_split(data, lengths=[0.7,0.1,0.2])

batch_size = 128

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)



## Model

Most suggestions ive seen involve LSTM, probably worth looking into that more

In [ ]:
# This initial implimentation is all from claude AI
# Going to try and get this working a bit and understand it
# And will then update

class ParameterPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, 
                           hidden_size=64, 
                           num_layers=2, 
                           batch_first=True)
        self.fc = nn.Linear(64, 1)
    
    def forward(self, x):
        # Reshape input to [batch, sequence, features]
        x = x.unsqueeze(-1)  # Add feature dimension: [batch, 4000, 1]
        lstm_out, _ = self.lstm(x)
        # Use last output for prediction
        last_out = lstm_out[:, -1, :]
        return self.fc(last_out).squeeze()

# Training Loop
Wrote this last year for the DSML exam, might need to be modified

In [ ]:
def trainModel(model, optimizer, loss_fcn, n_epochs, train_dloader, val_dloader, start_epoch = 0, patience = 3):
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    best_val_epoch = 0

    for epoch in range(start_epoch, start_epoch + n_epochs):
        model.train()
        tloss, vloss = 0, 0
        true_preds, counts = 0., 0
        for X_train, y_train in tqdm(train_dloader, desc='Epoch {}, training'.format(epoch+1)):
            optimizer.zero_grad()
            pred = model(X_train)
            loss = loss_fcn(pred, y_train.flatten())            
            tloss += loss.item()
            loss.backward()
            optimizer.step()
            true_preds+= (pred.argmax(dim = -1)==y_train.flatten()).sum()
            counts += len(y_train)

        train_acc = true_preds/counts
        train_accs.append(train_acc)
            
        model.eval()
        with torch.no_grad():
            vtrue_preds, vcount = 0., 0
            for X_valid, y_valid in tqdm(val_dloader, desc='Epoch {}, validation'.format(epoch+1)):
                pred = model(X_valid)
                loss = loss_fcn(pred, y_valid.flatten())
                vloss += loss.item()
                vtrue_preds += (pred.argmax(dim = -1)==y_valid.flatten()).sum()
                vcount += len(y_valid)
        val_acc = vtrue_preds/vcount
        val_accs.append(val_acc)

        print(f"[Epoch {epoch+1:2d}] Training accuracy: {train_acc*100.0:05.2f}%, Validation accuracy: {val_acc*100.0:05.2f}%")
        train_losses.append(tloss/len(train_data))
        val_losses.append(vloss/len(val_data))

        if val_acc > max(val_accs[:-1], default=0):
            print("New best validation performance")
            best_val_epoch = epoch

        elif best_val_epoch <= epoch - patience:
            print('No improvement in validation accuracy in last {} epochs'.format(patience))
            break
        

        
    return train_losses, val_losses

# Main Loop

In [ ]:
model = ParameterPredictor()
lossfcn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainModel(model, optimizer, lossfcn, 10, train_loader, val_loader)

random pycbc stuff

In [ ]:
from pycbc.waveform import get_td_waveform

hp, hc = get_td_waveform(approximant="IMRPhenomD", mass1=10,mass2=10,spin1z=0.9,delta_t=1/4096,f_lower=40)

print(hp.sample_times)